# Decoy Effect Model - Version 0.0

## Model Parameters

In [ ]:
import sys
import math
import random
import numpy as np
import pandas as pd
import mesa
import matplotlib.pyplot as plt
import matplotlib.colors as mplc

# Global Functions ################################################################

def diskmask(shape, radius):
    """ Produces a circular mask in the center of a grid
        Args:
            shape: of array grid
            radius: radius of mask
    """
    from skimage.draw import disk
    
    mask = np.zeros(shape)    
    rr, cc = disk((shape[0]/2, shape[0]/2), radius)
    mask[rr, cc] = 1
    
    return mask


def euclid(pos_1, pos_2):
    """ Get the Euclidean distance between two points
        Args:
            pos_1, pos_2: Coordinate tuples
    """
    return math.sqrt((pos_1[0] - pos_2[0])**2 + (pos_1[1] - pos_2[1])**2)


def event_probability(t, t_mean, t_std):
    """ Get the probability of an event from a cdf distribution with mean, std mean
        Args:
            t:      time variable value (eg. age)
            t_mean: mean time of event (eg. life span) 
        
    """
    from scipy.stats import norm
    return norm.cdf(t, loc=t_mean, scale=t_std)


def laplacian9p(agent, typ):
    """
    Discrete Laplace operator using a nine-point stencil (kernel):

          | 0.25 0.50 0.25 |
    D^2 = | 0.50 -3.0 0.50 | *(1/3)
          | 0.25 0.50 0.25 |

    We can define a factor = (0.5/(|x - x_0| + |y - y_0|) which is 
     - 0.25 for diagonal elements
     - 0.50 for non-diagonal elements
     - undefined for center element (irrelevant)

    """

    # Calculate discrete Laplacian in Moore's neighborhood
    norm = 1/3
    dif = -3*norm*agent.amount

    # Good way but it doesn't work well in the edges of the torus
    #neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
    #for neighbor in neighbors:
    #    if type(neighbor) is type(agent):        
    #        factor = 1/(np.abs(neighbor.pos[0] - self.pos[0]) + np.abs(neighbor.pos[1] - self.pos[1]))
    #        dif += 0.5*norm*factor*neighbor.amount

    neighbors = [(-1, -1),  (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for dpos in neighbors:
        den = np.abs(dpos[0]) + np.abs(dpos[1])
        pos = ((agent.pos[0] + dpos[0])% agent.model.width, 
               (agent.pos[1] + dpos[1])% agent.model.height)
        val = agent.model.get_agent(pos, typ).amount
        dif += 0.5*norm*val/den

    return dif
    
    
def sigmoid(x, mu, K):
    """ Response function: 
        Args:
            mu: response is linear with slope 'mu' for x small (x->0) 
            K: response saturates at value K for x large (x->inf)  
    """
    return K*(mu*x / (mu*x + K))


def animate(frames, nframes, cmap, norm, moviename, fps = 30):
        
    import matplotlib.animation as animation

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure( figsize=(8,8) )
    
    snapshots = [ frames[i, :, :] for i in range(nframes) ]

    im = plt.imshow(snapshots[0], interpolation='nearest', aspect='auto', cmap=cmap, norm=norm)

    def animate_func(i):
        if i % fps == 0:
            print( '.', end ='' )
        im.set_array(snapshots[i])
        return [im]

    anim = animation.FuncAnimation(fig, animate_func, 
                                   frames = nframes, interval = 1000 / fps) 
    #plt.show()
    #anim.save( moviename, writer=animation.FFMpegWriter(fps=fps))
    #anim.save(moviename, fps=fps, extra_args=['-vcodec', 'libx264'])
    
    return(anim)
                     

In [ ]:
import agents

In [ ]:
class Params():
    """
    class of simulation parameters
    
    """
    def __init__(self, i = 0, filein = None):
        """
        creates set of parameters with default values
    
        """
        
        # Simulation 
        self.id = i                              # ID of the parameter set
        self.width = 100                         # Width of array grid
        self.height = 100                        # Height of array grid
        self.initial_population_0 = 100          # Initial population of SNVs
        self.initial_population_1 = 50           # Initial population of SHVs
        self.initial_population_2 = 10           # Initial population of RNVs
        self.number_of_steps = 100               # Time length of simulation (in ticks)
        self.tumor_radius = 10                   # Radius of initial tumor  
        
        # Dossage controls
        self.drug_dossage = 100                  # Drug dossage per site  
        self.drug_dossage_time = 1000            # Time period for new dossage (+/- 1%)

        # Cell dynamics 
        self.life_span = 100                     # Age of cells natural death (+/- 10%)
        self.division_age = 5                    # Base age of cell division (+/- 5%)
        self.fitness_cost_0 = 1                  # Fitness factor for SNVs (multiply division age)
        self.fitness_cost_1 = 1.5                # Fitness factor for SHVs (multiply division age)
        self.fitness_cost_2 = 2                  # Fitness factor for RNVs (multiply division age)
        self.replacement_prob = 0.1              # Probability of cell replacement  
        self.kill_cell = 5                       # Killing threshold of cell by the drug
        
        # Vesicle dynamics
        self.mv_prod_0 = 0                       # Background vesicule production per cell per tick
        self.mv_prod_drug = 1                    # Vesicule production per cell per tick per unit drug
        self.mv_prod_max = 10                    # Maximun vesicle production per cell per tick
        self.kill_mv = 0.1                       # Max drug per MV (MV is `killed` when saturated)
        self.diff_mv = 0.1                       # Rate of MV difussion (per tick per site)
        self.mv_max = 50                         # Maximun MVs in grid site (MV saturation)
        
        # Drug dynamics
        self.drug_abs_cell = 0.5                 # Rate of drug absorption per cell per tick
        self.drug_abs_mv = 0.1                   # Rate of drug absorption per vesicle per tick
        self.drug_decay = 0.1                    # Rate of drug decay per tick    
        self.diff_drug = 0.25                    # Rate of drug difussion (per tick per site)    
        self.drug_max = 50                       # Maximune drug in grid site (drug saturation)
        
        # if a parameter filename is given, read parameters from it (index i)
        if filein is not None:
            self.read_parameters_from_file(filein, i)
        

    def get_parameter(self, param):

        return(getattr(self, param))

    def set_parameter(self, param, value):

        setattr(self, param, value)

    def read_parameters_from_file(self, fil, i):
        
        parameters = vars(self)
        
        tbl = pd.read_csv(fil)
        
        defaulted = [p for p in parameters if p not in tbl]
        extra = [p for p in tbl if p not in parameters]
        
        if defaulted:
            print("Attention! parameters: <" + str(defaulted) + \
              "> were not found in parameter table. They will be assigned default values...")
        if extra:
            print("Attention! parameters: <" + str(extra) + \
              "> in parameter table are not valid. They will be ignored...")
        
        for col in tbl:
            if col in parameters:
                self.set_parameter(col, tbl[col].values[i])
        
        
    def read_parameters_from_tbl(self, tbl, i):
        
        parameters = vars(self)
        
        defaulted = [p for p in parameters if p not in tbl]
        extra = [p for p in tbl if p not in parameters]
        
        if defaulted:
            print("Attention! parameters: <" + str(defaulted) + \
              "> were not found in parameter table. They will be assigned default values...")
        if extra:
            print("Attention! parameters: <" + str(extra) + \
              "> in parameter table are not valid. They will be ignored...")
        
        for col in tbl:
            if col in parameters:
                self.set_parameter(col, tbl[col].values[i])
                
                

In [ ]:
## Cell Agent class

class Cell(mesa.Agent):
    """
    Cell agent
    
    """
    def __init__(self, unique_id, model, pos, phenotype, age = 0, drug=0):
        """
        Cell agent constructor:
        
        (*) pos = (x,y) position in grid
        (*) phenotype = 0: SNV: sensitive cell (wild type with normal-vesicularization) 
                        1: SHV: sensitive cell with hightened-vesicularization 
                                (produces vesicles in response to drug stress) 
                        2: RNV: drug resistant cell (normal-vesicularization)

        (*) drug = amount of drug toxicity, if (drug > MU_DRUG) bug dies
        age = age in ticks

        """
        super().__init__(unique_id, model)
        self.pos = pos
        self._pos = pos
        self.phenotype = phenotype
        self.drug = drug
        self.age = age
        self.alive = True
        return    

    
    def move(self):
        # move cell to a neighboring grid site
        moved = False
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        candidates = [ pos for pos in neighborhood if not self.model.occupied(pos)]
        
        if candidates:
            # if available locations pick random one
            new_pos = self.random.choice(candidates)
            self._pos = new_pos
            self.model.moved_agents.append(self)
            moved = True
        elif (np.random.random(1)[0] < self.model.params.replacement_prob):
            # if there aren't available locations pick a random neighbor to replace
            new_pos = self.random.choice(neighborhood)
            dead_agent = self.model.get_agent(new_pos, Cell)
            dead_agent.die()
            #print("Death by replacement: ID=" + str(dead_agent.unique_id))
            self._pos = new_pos
            self.model.moved_agents.append(self)
            moved = True
            
        return moved
    
            
    def life_cycle(self):
        
        if (self.alive):
            rep_age = self.model.div_ages[self.phenotype]
            life_span = self.model.params.life_span

            if (np.random.random(1)[0] < event_probability(self.age, life_span, life_span/10)):
                # random natural death for the bug's age
                # print("Death by old age: ID=" + str(self.unique_id) + '; age=' + str(self.age))
                self.die()
            elif (np.random.random(1)[0] < event_probability(self.age, rep_age, rep_age/20)):
                # random replication for the cell's age; try to move out of pos and divide
                pos = self.pos
                if (self.move()):
                    # if it can move, then divides an place daughter in original pos
                    # print("Cell division: ID=" + str(self.unique_id) + '; age=' + str(self.age))
                    self.age = 0
                    self.drug = self.drug/2   # divide drug between childs
                    self.divide(pos, self.phenotype, self.drug)
        return

    
    def drug_in(self):
        # cell takes drug (if phenotype is SNV or SHV)
        Ac = self.model.params.drug_abs_cell
        
        if (self.phenotype < 2 and self.alive):
            # find amount of drug in cell location
            drug_patch = self.model.get_agent(self.pos, Drug)
            # dose is capped at 'Ac'
            dose = np.min([drug_patch.amount, Ac])
            # cell takes the drug
            self.drug += dose
            drug_patch.amount -= dose
            if self.drug > self.model.params.kill_cell:
                # print("Death by drug: ID=" + str(self.unique_id))
                self.die()
    
    
    def vesiculate(self):
        # cell vesiculates
        NU_0 = self.model.params.mv_prod_0
        NU_D = self.model.params.mv_prod_drug
        NU_MAX = self.model.params.mv_prod_max
        MAX_MVS_SITE = self.model.params.mv_max
        
        mv_patch = self.model.get_agent(self.pos, MV)
        # if cell is SHV
        if (self.phenotype == 1):
            # finds amount of drug in location
            drug_patch = self.model.get_agent(self.pos, Drug)
            # Vesicle production is a response to drug amount in location
            mvs = NU_0 + sigmoid(drug_patch.amount, NU_D, NU_MAX) 
        else:
            # background vesicle production
            mvs = NU_0
        # updates amount of vesicles in site, capped by 'MAX_MVS_SITE'
        mv_patch.amount = np.min([mv_patch.amount + mvs, MAX_MVS_SITE])
        # print("Vesiculation: " + str(mv_patch.amount) + " at: " + str(mv_patch.pos))
    
    
    def divide(self, pos, phenotype, drug):
        #print("Cell divide: ID=" + str(self.unique_id) + " at:" + str(self.pos))
        self.model.max_id += 1
        new_cell = Cell(self.model.max_id, self.model, pos, phenotype, drug)
        #print("Daughter: ID=" + str(new_cell.unique_id) + " at:" + str(new_cell.pos))
        self.model.born_agents.append(new_cell)
            

    def die(self):
        #print("Cell died: ID=" + str(self.unique_id) + " at:" + str(self.pos))
        self.alive = False
        if self not in self.model.dead_agents:
            self.model.dead_agents.append(self)
    
    
    def step(self):
        self.drug_in()
        self.life_cycle()

        
    def advance(self):
        if self.alive:
            self.vesiculate()
            self.age += 1
        
        
######################################################################################################

class MV(mesa.Agent):
    """
    Vesicle patch agent: it exist on each position and the 'amount' accounts for local concentration
    
    """
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = 0.0
        self.drug = 0.0
        self._nextAmount = 0.0
        
        
    def drug_in(self):
        # absorbtion of drug by MVs
        Av = self.model.params.drug_abs_mv
        MAX_DRUG = self.model.params.kill_mv
        
        if (self.amount > 0):
            drug_patch = self.model.get_agent(self.pos, Drug)
            # dose is capped at 'Av' per tick per vesicle
            dose = np.min([drug_patch.amount, Av*self.amount])
            if (dose > 0):
                # get number of vesicles that would saturate with this amount of drug 
                # leftover is the residual drug after saturating MVs
                num_mvs, leftover = divmod(self.drug + dose, self.amount*MAX_DRUG) 

                # if all vesicles in location saturate
                if (num_mvs > self.amount):
                    # the amount of drug to cap out all MVs
                    dose = self.amount*MAX_DRUG - self.drug
                    # and no MVs are left out
                    num_mvs = self.amount
                    leftover = 0
                # update amounts
                self.amount -= num_mvs
                self.drug = leftover
                drug_patch.amount -= dose
 

    def diffuse(self):
        # Calculate amount of change due to difussion
        return self.model.params.diff_mv * laplacian9p(self, MV)

    def step(self):
        # vesicle difussion using discrete Laplacian 
        self._nextAmount = self.amount + self.diffuse()
        
        
    def advance(self):
        """
        Set the state to the new computed state -- computed in step().
        """
        self.amount = self._nextAmount
        #print("MV: " + str(self.amount) + " at: " + str(self.pos))
        self.drug_in()


#####################################################################################################

class Drug(mesa.Agent):
    """
    Drug patch agent: it exist on each position and the 'amount' accounts for local concentration
    
    """
    def __init__(self, unique_id, model, pos, drug):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = drug
        self._nextAmount = 0.0

    
    def diffuse(self):
        # Calculate amount of change due to diffusion
        return self.model.params.diff_drug * laplacian9p(self, Drug)
    
    def step(self):
        # drug difussion using discrete Laplacian 
        self._nextAmount = self.amount + self.diffuse()
       
    
    def advance(self):
        """
        Set the state to the new computed state -- computed in step().
        """
        self.amount = self._nextAmount*math.exp(-self.model.params.drug_decay)
        

In [ ]:
class Decoy(mesa.Model):
    """
    Decoy Model
    """
    def __init__(self, params, verbose = False):
        """
        Create a new model 
        """

        # Set parameters
        self.params = params
        self.id = params.id
        self.width = params.width
        self.height = params.height
        self.time = 0
        self.last_dossage_time = 0
        self.verbose = verbose            # Print-monitoring
        self.dead_agents = list()         # List of dead agents per step
        self.moved_agents = list()        # List of moved agents per step
        self.born_agents = list()         # List of born agents per step
        
        
        # local variables 
        initial_population = list((params.initial_population_0,
                                   params.initial_population_1,
                                   params.initial_population_2))
                                  
        # division age
        self.div_ages = list((params.division_age * params.fitness_cost_0,
                              params.division_age * params.fitness_cost_1,
                              params.division_age * params.fitness_cost_2))
         
        
        # scheduler for simultaneous activation of all the agents
        self.schedule = mesa.time.SimultaneousActivation(self)
        # model grid allows for several agents per site
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)
        
        # NEEDS WORK HERE!!!!!
        # schedule data collection
        mr={"Populations": self.compute_populations}
        self.datacollector = mesa.DataCollector(model_reporters= mr)

        agent_id = 0
        
        # Create field agents: drug and mv
        for _, x, y in self.grid.coord_iter():
            drug = Drug(agent_id, self, (x, y), 0)
            agent_id += 1
            self.grid.place_agent(drug, (x, y))
            self.schedule.add(drug)
            mv = MV(agent_id, self, (x, y))
            agent_id += 1
            self.grid.place_agent(mv, (x, y))
            self.schedule.add(mv)
    
        # add drug to the system
        self.add_drug()
        
        # Create cell agents
        def select_in_mask(w, h, mask):
            # select random (x,y) unocuppied coordinates in mask
            x = self.random.randrange(w)
            y = self.random.randrange(h)
            while (mask[x,y] == 0 or self.occupied((x,y))):
                x = self.random.randrange(w)
                y = self.random.randrange(h)
            return (x,y)
        
        # tumor mask (M is its total size)
        #mask = np.ones((self.width, self.height))
        mask = diskmask((self.width, self.height), self.params.tumor_radius)
        M = np.sum(mask)
        
        # N is the total initial population
        N = np.sum(initial_population)
        for typ in (0, 1, 2):
            if (N > M):
                # if tumor size is too small, renormalizes populations to fit
                initial_population[typ] = (initial_population[typ]/N)*M
            # makes sure there is at least one cell of each kind
            initial_population[typ] = int(np.max([initial_population[typ], 1]))
            for i in range(initial_population[typ]):
                # find a random location
                x, y =  select_in_mask(self.width, self.height, mask)
                             
                # pick a random initial age within reproductive age
                age = np.random.randint(self.div_ages[typ], size=1)[0]
                cell = Cell(agent_id, self, (x, y), typ, age, 0)
                agent_id += 1
                self.grid.place_agent(cell, (x, y))
                self.schedule.add(cell)
        
        self.max_id = agent_id
        self.running = True
        self.datacollector.collect(self)
  

    def compute_populations(self):
        """
        Get population abundances for each phenotype
        """
        population = [0, 0, 0]
        for agent in self.schedule.agents:
            if type(agent) is Cell:
                population[agent.phenotype] += 1
        return population
    
    
    def cell_table(self):
        """
        Get table of all living cell properties
        """
        cell_props = {"time": [], "cell_id": [], "phenotype": [],
                      "age": [], "x": [], "y": [], "drug": []}
        for agent in self.schedule.agents:
            if type(agent) is Cell:
                if agent.alive:
                    cell_props["time"].append(self.time)
                    cell_props["cell_id"].append(agent.unique_id)
                    cell_props["phenotype"].append(agent.unique_id)
                    cell_props["age"].append(agent.age)
                    cell_props["x"].append(agent.pos[0])
                    cell_props["y"].append(agent.pos[1])
                    cell_props["drug"].append(agent.drug)
        return pd.DataFrame(cell_props)
    
    
    def get_agent(self, pos, typ):
        # get ALL agents of type typ in this location
        agents = self.get_agents(pos, typ)
        return agents[0]
    
    
    def get_agents(self, pos, typ):
        # get ALL agents of type typ in this location
        agents_out = list()
        agents = self.grid.get_cell_list_contents([pos])
        for agent in agents:
            if type(agent) is typ:
                agents_out.append(agent)
        return agents_out
        
        
    def occupied(self, pos):
        # get if 'pos' is occupied by a cell agent
        agents = self.grid.get_cell_list_contents([pos])
        return any(isinstance(agent, Cell) for agent in agents)
    
    
    def add_drug(self):
        #drug_distribution = np.genfromtxt("drug-map.txt")
        drug_distribution = self.params.drug_dossage*np.ones((self.width, self.height))
        for _, x, y in self.grid.coord_iter():
            dose = drug_distribution[x, y]
            drug = self.get_agent((x,y), Drug)
            drug.amount = np.min([drug.amount + dose, self.params.drug_dossage])
            
    
    def step(self):
                     
        # Time step of the model 
        self.time += 1
        self.schedule.step()  # runs all scheduled events
        
        # updates states of agents 
        # (this has to be done here because the schedule is simultaneous)
        # NOTE: This might be done using the agent.advance attribute, but not sure how yet 
        
        # clean out agents scheduled to move which are also scheduled to die
        # (e.g. an agent is scheduled to die by replacement after it was scheduled to move)
        self.moved_agents = [agent for agent in self.moved_agents if agent not in self.dead_agents]
                
        # refresh dead agents in model schedule and grid
        self.dead_agents = list(set(self.dead_agents)) 
        for agent in self.dead_agents:
            #print("died: ID:" + str(agent.unique_id) + "; pos = " + str(agent.pos))
            self.schedule.remove(agent)
            self.grid.remove_agent(agent)
            #self.dead_agents.remove(agent)
        self.dead_agents = list()

        # refresh born agents in model schedule and grid
        self.born_agents = list(set(self.born_agents)) 
        for agent in self.born_agents:   
            if (len(self.get_agents(agent.pos, Cell)) == 1):
                # if new cell is the only one in 'pos', places and schedules it
                #print("Born, " + str(self.time) + " ID:" + str(agent.unique_id) + "; pos = " + str(agent.pos))
                self.grid.place_agent(agent, agent.pos)
                self.schedule.add(agent)
            else:
                # if another agent is already in place, abort cell birth
                #print("Aborted, " + str(self.time) + "; ID:" + str(agent.unique_id) + "; pos = " + str(agent.pos))
                self.grid.place_agent(agent, agent.pos)
                self.grid.remove_agent(agent)
            #self.born_agents.remove(agent)
        self.born_agents = list()
        
        # refresh moved agents in model grid
        self.moved_agents = list(set(self.moved_agents)) 
        for agent in self.moved_agents: 
            if self.occupied(agent._pos):
                # if position is already taken (eg. two agents moved to the same spot) kills agent
                #print("Killed, " + str(self.time) + "; ID:" + str(agent.unique_id) + "; pos = " + str(agent.pos)+ "; _pos = " + str(agent._pos))
                self.schedule.remove(agent)
                self.grid.remove_agent(agent)
            else: 
                # move agent to the grid position assigned in agent.move()
                #print("Moved, " + str(self.time) + "; ID:" + str(agent.unique_id) + "; pos = " + str(agent.pos)+ "; _pos = " + str(agent._pos))
                self.grid.move_agent(agent, agent._pos)
            #self.moved_agents.remove(agent)
        self.moved_agents = list()
        
        # adds more drug to the system
        dossage_time = self.time - self.last_dossage_time
        if (np.random.random(1)[0] < event_probability(dossage_time, 
                                                       self.params.drug_dossage_time, 
                                                       self.params.drug_dossage_time/100)):
            self.last_dossage_time = self.time
            self.add_drug()
           
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print([self.schedule.time, self.compute_populations()], end="\r")

            
    def get_arrays(self):
        
        cellgrid = np.zeros((self.width, self.height))
        mvgrid = np.zeros((self.width, self.height))
        druggrid = np.zeros((self.width, self.height))
    
        for pos in self.grid.coord_iter():
            agents, x, y = pos

            cell = [agent.phenotype for agent in agents if type(agent) is Cell]
            if (len(cell)==0):
                cell = [-1]
            mv = [agent.amount for agent in agents if type(agent) is MV]
            drug = [agent.amount for agent in agents if type(agent) is Drug]

            cellgrid[y, x] = cell[0]
            mvgrid[y, x] = mv[0]
            druggrid[y, x] = drug[0]
        
        return (cellgrid, mvgrid, druggrid)
    
    
    def collect_data(self, i, cell_grid, mv_grid, drug_grid):
        
        cellgrid, mvgrid, druggrid = self.get_arrays()
        cell_grid[i, :, :] = cellgrid
        mv_grid[i, :, :] = mvgrid
        drug_grid[i, :, :] = druggrid 
       
        return (cell_grid, mv_grid, drug_grid)
        
        
    def run_model(self, fileout):
        
        steps = self.params.number_of_steps
        
        premature_finish = False
        
        cell_grid = np.zeros((steps, self.width, self.height))
        mv_grid = np.zeros((steps, self.width, self.height))
        drug_grid = np.zeros((steps, self.width, self.height))
        t_series = pd.DataFrame({"Step":[],"SNVs":[],"SHVs":[],"RNVs":[]})
        
        N0, N1, N2 = list(self.compute_populations())
        t_series = pd.concat([t_series, pd.DataFrame({"Step" : [0], 
                                                      "SNVs": [N0], 
                                                      "SHVs": [N1], 
                                                      "RNVs": [N2]})])
        cell_table = self.cell_table()
        cell_grid, mv_grid, drug_grid = self.collect_data(0, cell_grid, mv_grid, drug_grid)
        
        if self.verbose:
            print( "Initial Population: %s" % ((N0, N1, N2),))

        for i in range(steps - 1):
            
            self.step()
            
            N0, N1, N2 = list(self.compute_populations())
            
            if (np.sum([N0, N1, N2]) <= 0):
                print('Whole population died!!!')
                premature_finish = True
                break
                
            if (np.sum([N0, N1, N2]) > self.width*self.height*0.8):
                print('Population grew too big!!!')
                premature_finish = True
                break
            
            t_series = pd.concat([t_series, pd.DataFrame({"Step" : [i + 1],
                                                          "SNVs": [N0],
                                                          "SHVs": [N1],
                                                          "RNVs": [N2]})])
            cell_table = pd.concat([cell_table, self.cell_table()]) 
            cell_grid, mv_grid, drug_grid = self.collect_data(i + 1, cell_grid, mv_grid, drug_grid)
        
        
        if self.verbose:
            print("#################")
            print("Final Population: %s" % ((N0, N1, N2),))
            
            
        # reduces outputs to actual final 
        if premature_finish:
            cell_grid = cell_grid[0:i, :, :]
            mv_grid = mv_grid[0:i, :, :]
            drug_grid = drug_grid[0:i, :, :]
        
        # saves output into a file
        np.savez_compressed(fileout,
                            cell_grid=cell_grid,
                            mv_grid=mv_grid,
                            drug_grid=drug_grid,
                            t_series=t_series.to_numpy(),
                            cell_table=cell_table.to_numpy())
            
        return (cell_grid, mv_grid, drug_grid, t_series, cell_table)    
            

## OPTION 1: read parameters from file

In [ ]:
params = Params(0, 'parameters.csv')

## OPTION 2: set parameters manually

In [ ]:
params = Params(0)

# Simulation Parameters
params.set_parameter("width", 100)                         # Width of array grid
params.set_parameter("height", 100)                        # Height of array grid
params.set_parameter("initial_population_0", 100)          # Initial population of SNVs
params.set_parameter("initial_population_1", 50)           # Initial population of SHVs
params.set_parameter("initial_population_2", 10)           # Initial population of RNVs
params.set_parameter("number_of_steps", 100)               # Time length of simulation (in ticks)
        
# Dossage controls
params.set_parameter("drug_dossage", 100)                  # Drug dossage per site  
params.set_parameter("drug_dossage_time", 1000)            # Time period for new dossage (+/- 1%)

# Cell dynamics 
params.set_parameter("life_span", 100)                     # Age of cells natural death (+/- 10%)
params.set_parameter("division_age", 5)                    # Base age of cell division (+/- 5%)
params.set_parameter("fitness_cost_0", 1)                  # Fitness factor for SNVs (multiply division age)
params.set_parameter("fitness_cost_1", 1.5)                # Fitness factor for SHVs (multiply division age)
params.set_parameter("fitness_cost_2", 2)                  # Fitness factor for RNVs (multiply division age)
params.set_parameter("replacement_prob", 0.1)              # Probability of cell replacement  
params.set_parameter("kill_cell", 5)                       # Killing threshold of cell by the drug
        
# Vesicle dynamics
params.set_parameter("mv_prod_0", 0)                       # Background vesicule production per cell per tick
params.set_parameter("mv_prod_drug", 1)                    # Vesicule production per cell per tick per unit drug
params.set_parameter("mv_prod_max", 10)                    # Maximun vesicle production per cell per tick
params.set_parameter("kill_mv", 0.1)                       # Max drug per MV (MV is `killed` when saturated)
params.set_parameter("diff_mv", 0.1)                       # Rate of MV difussion (per tick per site)
params.set_parameter("mv_max", 50)                         # Maximun MVs in grid site (MV saturation)

# Drug dynamics
params.set_parameter("drug_abs_cell", 0.5)                 # Rate of drug absorption per cell per tick
params.set_parameter("drug_abs_mv", 0.1)                   # Rate of drug absorption per vesicle per tick
params.set_parameter("drug_decay", 0.1)                    # Rate of drug decay per tick    
params.set_parameter("diff_drug", 0.25)                    # Rate of drug difussion (per tick per site)    
params.set_parameter('drug_max', 50)                       # Maximune drug in grid site (drug saturation)

## Run single simulation

In [ ]:
simid = "sim_" + "{:04d}".format(params.id) 
decoy_model = Decoy(params, verbose = True)
outfile = "simulations/" + simid + ".npz"
cell_grid, mv_grid, drug_grid, time_series, cell_table = decoy_model.run_model(outfile)

## Run a batch, using parameter file

In [ ]:
params_tbl = pd.read_csv('parameters.csv')
params = Params()

N = len(params_tbl.index)
for i in range(N):
    
    params.read_parameters_from_tbl(params_tbl , i)
    simid = "sim_" + "{:04d}".format(params.id)
    outfile = "simulations/" + simid + ".npz"
    
    print(">>> Running Simulation: " + simid " ... "+ str(i+1) + "/" + str(N), end="\r")
    
    decoy_model = Decoy(params, verbose = False)
    cell_grid, mv_grid, drug_grid, time_series, cell_table = decoy_model.run_model(outfile)
    
    

## Retreive data saved in previous run

In [ ]:
params_tbl = pd.read_csv('parameters.csv')

i = 0    # index of result of interest
params.read_parameters_from_tbl(params_tbl , i)
simid = "sim_" + "{:04d}".format(params.id) 
outfile = "simulations/" + simid + ".npz"

aux = np.load(outfile)
cell_grid = aux['cell_grid'] 
mv_grid = aux['mv_grid'] 
drug_grid = aux['drug_grid'] 
t_series = pd.DataFrame(aux['t_series'], columns = ['Step','SNVs','SHVs', 'RNVs'])
cell_table = pd.DataFrame(aux['cell_table'], columns = ['time','cell_id','phenotype', 
                                                        'age', 'x', 'y', 'drug'])


## Simple visualization

In [ ]:
# We need `notebook` for the anim to render in the notebook
%matplotlib notebook

# check is output is consistent
N = len(time_series.SNVs)
aux = []
for i in range(N):
    a = cell_grid[i, :, :]
    aux.append(np.sum(a == 0))
    
snvs = pd.DataFrame({"step": [i for i in range(N)], 
                     "sche": list(time_series.SNVs),
                     "arra": aux})

plt.plot(snvs["sche"], snvs["arra"], 'r.')
plt.plot(snvs["sche"], snvs["sche"], 'k-')

In [ ]:
cell_bins = [-1, 0, 1, 2, 3]
cell_colors = ("#000000", "#2b83ba", "#1a9641", "#d7191c")

cmap = mplc.ListedColormap(cell_colors)
norm = mplc.BoundaryNorm(cell_bins, len(cell_bins))

#plt.imshow(cells[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()

anim = animate(cell_grid, N, cmap, norm,  "cells_"+ simid + ".mp4")

In [ ]:
mv_bins = np.linspace(0, np.max(mv_grid), 11, endpoint=True)
#mv_bins = np.linspace(0, MAX_MVS_SITE, 11, endpoint=True)
mv_colors = ("#ffffff", "#f7fcfd", "#e5f5f9", "#ccece6", "#99d8c9",
             "#66c2a4", "#41ae76", "#238b45", "#006d2c", "#00441b", "#000000")

cmap = mplc.ListedColormap(mv_colors)
norm = mplc.BoundaryNorm(mv_bins, len(mv_bins))

#plt.imshow(mv_grid[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()

anim = animate(mv_grid, N, cmap, norm,  "mvs_"+ simid + ".mp4")

In [ ]:
drug_bins = np.linspace(0, np.max(drug_grid), 11, endpoint=True)
#drug_bins = np.linspace(0, MAX_DRUG_SITE, 11, endpoint=True)
drug_colors = ("#ffffff", "#ffffcc", "#ffeda0", "#fed976", "#feb24c", 
               "#fd8d3c", "#fc4e2a", "#e31a1c", "#bd0026", "#800026", "#000000")

cmap = mplc.ListedColormap(drug_colors)
norm = mplc.BoundaryNorm(drug_bins, len(drug_bins))

#plt.imshow(drug_grid[-1, :, :], interpolation='nearest', cmap=cmap, norm=norm)
#plt.show()
anim = animate(drug_grid, N, cmap, norm,  "drugs_"+ simid + ".mp4")

# Using Mesa visualization tools

In [ ]:
def agent_portrayal(agent):
    
    cell_colors = ("#2b83ba", "#1a9641", "#d7191c")

    mv_bins = np.linspace(0, MAX_MVS_SITE, 11, endpoint=True)
    mv_colors = ("#ffffff", "#f7fcfd", "#e5f5f9", "#ccece6", "#99d8c9",
                 "#66c2a4", "#41ae76", "#238b45", "#006d2c", "#00441b", "#000000")

    drug_bins = np.linspace(0, MAX_DRUG_SITE, 11, endpoint=True)
    drug_colors = ("#ffffff", "#ffffcc", "#ffeda0", "#fed976", "#feb24c", 
                   "#fd8d3c", "#fc4e2a", "#e31a1c", "#bd0026", "#800026", "#000000")

    if agent is None:
        return

    if type(agent) is Cell:
        return {"Color": cell_colors[agent.phenotype],
                "Shape": "rect",
                "Filled": "true",
                "Layer": 2,
                "w": 1,
                "h": 1, }
    
    elif type(agent) is MV:
        return {"Color": mv_colors[np.digitize(agent.amount, mv_bins, right=True)],
                "Shape": "rect",
                "Filled": "true",
                "Layer": 1,
                "w": 1,
                "h": 1, }
    
    elif type(agent) is Drug:
        return {"Color": drug_colors[np.digitize(agent.amount, drug_bins, right=True)],
                "Shape": "rect",
                "Filled": "true",
                "Layer": 0,
                "w": 1,
                "h": 1, }

    return {}


canvas_element = mesa.visualization.CanvasGrid(agent_portrayal, WIDTH , HEIGHT , 600 , 600 )

#chart_element = mesa.visualization.ChartModule([{"Label": "Agent", "Color": "#AA0000"}])

server = mesa.visualization.ModularServer(Decoy, 
                                          #[canvas_element, chart_element],
                                          [canvas_element],
                                          "Decoy Model V.0.0",
                                         {"width":WIDTH, 
                                          "height":HEIGHT, 
                                          "initial_population": INITIAL_POPULATION})

In [ ]:
server.launch()

# MISC helping code

## the event probability function

In [ ]:
################################################################################################

from scipy.stats import norm
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
t = np.linspace(0, 2*REPLICATION_AGE, 1000)
ax.plot(t, event_probability(t, REPLICATION_AGE, REPLICATION_AGE/20), color = 'b')
ax.axvline(x = REPLICATION_AGE, color = 'r', ls='--')
ax.axvline(x = (REPLICATION_AGE)*(0.95), color = 'r', ls=':')
ax.axvline(x = (REPLICATION_AGE)*(1.05), color = 'r', ls=':')
ax.axhline(y = 1, color = 'k', ls=':')
ax.axhline(y = 0, color = 'k', ls=':')
ax.axhline(y = 0.5, color = 'k', ls=':')

## the response Sigmoid function

In [ ]:
fig, ax = plt.subplots(1, 1)
x = np.linspace(0, 5*MAX_MVS, 1000)
ax.plot(x, sigmoid(x, NU_D, MAX_MVS), color = 'b')
ax.plot(x, NU_D*x, color = 'k')
ax.axhline(y = MAX_MVS, color = 'k', ls=':')
ax.axhline(y = NU_0, color = 'k', ls=':')
ax.axhline(y = 0, color = 'k', ls=':')
ax.axvline(x = 0, color = 'r', ls=':')
ax.set_ylim([0, MAX_MVS*1.2])

## animation example

In [ ]:
# Usually we use `%matplotlib inline`. However we need `notebook` for the anim to render in the notebook.
%matplotlib notebook

import random
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import matplotlib.animation as animation


fps = 30
#snapshots = [ np.random.rand(5,5) for _ in range( nSeconds * fps ) ]
snapshots = [ cell_grid[i, :, :] for i in range(100) ]

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure( figsize=(8,8) )

im = plt.imshow(snapshots[0], interpolation='nearest', aspect='auto', cmap=cmap, norm=norm)

def animate_func(i):
    if i % fps == 0:
        print( '.', end ='' )

    im.set_array(snapshots[i])
    return [im]

anim = animation.FuncAnimation(fig, 
                               animate_func, 
                               frames = 100,
                               interval = 1000 / fps)

#anim.save('test_anim.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

print('Done!')


In [ ]:
cell_grid.shape


In [ ]:
cellgrid = np.zeros((decoy_model.grid.width, decoy_model.grid.height))
mvgrid = np.zeros((decoy_model.grid.width, decoy_model.grid.height))
druggrid = np.zeros((decoy_model.grid.width, decoy_model.grid.height))

for pos in decoy_model.grid.coord_iter():
    agents, x, y = pos
    
    cell = [agent.phenotype for agent in agents if type(agent) is Cell]
    if (len(cell)==0):
        cell = [-1]
    mv = [agent.amount for agent in agents if type(agent) is MV]
    drug = [agent.amount for agent in agents if type(agent) is Drug]
    
    cellgrid[y][x] = cell[0]
    mvgrid[y][x] = mv[0]
    druggrid[y][x] = drug[0]
    